In [1]:
import tensorflow as tf
from keras.layers import Dense
print ("TensorFlow version: " + tf.__version__)

TensorFlow version: 1.12.0


Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from datetime import datetime

In [4]:
pc_df = pd.read_csv("PatientCorePopulatedTable.csv",na_values = ['Unknown'], infer_datetime_format = True, parse_dates = ['PatientDateOfBirth'])

In [5]:
pc_df.head(5)

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty
0,3A3C2AFB-FFFA-4E69-B4E6-73C1245D5D12,Male,1975-01-04 14:49:59.587,White,Single,NaN,15.6
1,801AFB51-036F-40E3-BDFE-FED4844BE275,Male,1964-09-06 13:15:43.043,White,NaN,English,13.23
2,366B0CC6-18AC-45DD-9AD4-BE884FE3A299,Female,1953-01-14 06:00:19.330,White,Married,English,12.11
3,DBB78149-D86C-435E-82C4-341999FD0719,Female,1986-04-28 12:42:02.007,NaN,NaN,NaN,95.8
4,2C6269F4-71F8-4C07-A905-C08E9A3524C2,Female,1963-08-26 10:57:55.183,White,NaN,English,18.08


In [6]:
pc_df.loc[pc_df['PatientGender'] == 'female', 'PatientGender'] = 'Female'
pc_df.loc[pc_df['PatientRace'] == 'Aslan', 'PatientRace'] = 'Asian'
pc_df.loc[pc_df['PatientLanguage'] == 'Engrish', 'PatientLanguage'] = 'English'
pc_df.loc[pc_df['PatientPopulationPercentageBelowPoverty'] == 'text', 'PatientPopulationPercentageBelowPoverty'] = ''

In [7]:
pc_df['PatientGender'] = pc_df.PatientGender.astype('category')
pc_df['PatientRace'] = pc_df.PatientRace.astype('category')
pc_df['PatientMaritalStatus'] = pc_df.PatientMaritalStatus.astype('category')
pc_df['PatientLanguage'] = pc_df.PatientLanguage.astype('category')
pc_df['PatientPopulationPercentageBelowPoverty'] = (pd.to_numeric(pc_df['PatientPopulationPercentageBelowPoverty'], errors='coerce'))

In [8]:
from datetime import datetime
pc_df["PatientAge"] = (pd.Timestamp(datetime.now()) - pc_df["PatientDateOfBirth"]).astype('<m8[Y]')

In [10]:
# pc_df.drop(pc_df.index[0], inplace=False)
pc_df.drop(pc_df.index[4199], inplace=True)
pc_df.drop(pc_df.index[7248], inplace=True)

In [ ]:
# pc_df = pc_df.drop(['PatientDateOfBirth'], axis = 1)

In [12]:
def summerize_data(pc_df):
    for column in pc_df.columns:
        print (column)
        if pc_df.dtypes[column] == np.object: # Categorical data
            print (pc_df[column].value_counts())
        else:
            print (pc_df[column].describe()) 
        
        print ('\n')
    
summerize_data(pc_df)

PatientID
AFA3A6E6-B856-448A-995F-BD518309A788    1
C771BFC7-ADAE-4BDE-B62E-560990B90790    1
D4F70788-1163-4C33-89B2-03C896FC55FE    1
707386F7-5433-4677-AE52-26BB13ECD86D    1
FF772443-075D-46A0-AA8B-72D987971E9F    1
839A0426-AFCF-4795-9F6E-436CCCCF1318    1
839A4F07-D559-4F44-9BEE-93D1D717793D    1
3107E3BC-FF10-4288-815C-19F4902C8FDC    1
FE494774-4C27-45EA-B87A-CA7CE0609B30    1
F4B1AD6B-0D76-48D0-8413-01140D5AA0F3    1
248BD5C5-3849-4458-B8F9-70B81941B37F    1
0C57954D-D0C9-494C-AAB2-AE340E72A4E7    1
A886C6E8-E77C-45C1-B8D3-CE45FB5C73ED    1
B2DE3750-9CE0-4434-B7F5-630AB9B3CFDB    1
14ACE1F3-208E-4263-B034-4F894B0E13F7    1
89BC8490-5984-414B-9467-AF90354DF654    1
D1CB75AA-CEAD-4603-9E53-89149A0E6441    1
B3EBFD9A-7EA9-486F-9B0A-3589BCB5861E    1
5A2EE954-42DB-4210-A944-4E4784BB1F35    1
F728D6CC-D2D0-4ADA-8D19-FBE9DE4D932B    1
F5EE8716-1B5D-41E9-BDCB-A97EA2D8C1D9    1
931918E9-5D70-4161-9FFC-8F0550F3F86B    1
65F82D78-0B5A-4689-812D-50B06842BF65    1
0BC915D6-E8B0-470A-ABFB-

In [13]:
print (list(pc_df))
pc_df.reset_index()

['PatientID', 'PatientGender', 'PatientDateOfBirth', 'PatientRace', 'PatientMaritalStatus', 'PatientLanguage', 'PatientPopulationPercentageBelowPoverty', 'PatientAge']


,index,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty,PatientAge
0,0,3A3C2AFB-FFFA-4E69-B4E6-73C1245D5D12,Male,1975-01-04 14:49:59.587,White,Single,NaN,15.60,44.0
1,1,801AFB51-036F-40E3-BDFE-FED4844BE275,Male,1964-09-06 13:15:43.043,White,NaN,English,13.23,54.0
2,2,366B0CC6-18AC-45DD-9AD4-BE884FE3A299,Female,1953-01-14 06:00:19.330,White,Married,English,12.11,66.0
3,3,DBB78149-D86C-435E-82C4-341999FD0719,Female,1986-04-28 12:42:02.007,NaN,NaN,NaN,95.80,32.0
4,4,2C6269F4-71F8-4C07-A905-C08E9A3524C2,Female,1963-08-26 10:57:55.183,White,NaN,English,18.08,55.0
5,5,07E4EDF1-3AC8-4A71-AC06-F5DDDD8032BA,Male,1958-11-08 01:27:12.597,White,Single,English,12.38,60.0
6,6,90A956B2-AAE0-4D39-8A5E-680B36E1EA46,Male,1957-12-27 14:00:00.410,African American,Single,English,19.47,61.0
7,7,00F8A4BE-3CE3-466A-A90D-0A246D80CE57,Female,1925-03-27 18:58:51.580,White,Divorced,Icelandic,13.47,93.0
8,8,4731F86A-9225-424F-9944-AEF4001D0C55,Male,1953-10-07 11:56:58.470,African American,Separated,English,19.97,65.0
9,9,4CF45EFD-ACB3-4199-A974-403F25D219F2,Female,1969-07-02 04:52:46.833,White,Married,Spanish,15.55,49.0


In [1]:
# # Encode the categorical features as numbers
# def number_encode_features(pc_df):
#     result = pc_df.copy()
#     encoders = {}
#     for column in result.columns:
#         if result.dtypes[column] == np.object:
#             encoders[column] = preprocessing.LabelEncoder()
#             result[column] = encoders[column].fit_transform(result[column])
#     return result, encoders


# # Calculate the correlation and plot it
# encoded_data, _ = number_encode_features(pc_df)
# sns.heatmap(encoded_data.corr(), square=True)
# plt.show()

# encoded_data.tail(5)

In [ ]:
pc_df.to_csv('cleaned.txt')